In [8]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install farasapy keras-self-attention

In [10]:
import pandas as pd

arabic_data = pd.read_excel('/content/drive/MyDrive/Freelancing/project 11/uploaded-dataset/articles.xlsx')
arabic_data = arabic_data[["articles","summary"]]
arabic_data.head()

,articles,summary
0,و هو يعتبر متحف جديد العهد ، يقدم مجموعة من العروض الفنية و التشكيلية المتنوعة ، يظهر فنانين عرب معاصرين و تبرز إبداعاتهم الفريدة ، كما أنه يبرز الرؤية العربية عن الفن المعاصر و الفن الحديث العالم...,المتحف العربي للفن الحديث
1,يعتبر فيلاجيو مول من أهم المجمعات التجارية و أشدها جمالا في العالم العربي ، و ذلك لأنه تم بناؤه و تصميمه على هيئة مدينة البندقية الموجودة في إيطاليا ، و هذا ما جعله أمر شيق لكل أفراد العائلة ، و ل...,فيلاجيو مول
2,هو المنطقة البحرية الممتدة على مجموعة كيلومترات بشكل طولي على خليج الدوحة ؛ حيث تطل على الكورنيش مجموعة من المعالم المهمة ، مثل : متحف الفن الإسلامي ، وحديقة الرميلة و فندق الشيراتون الموجود على ه...,كورنيش الدوحة
3,يعتبر هذا المنتجع من أجمل الأماكن الساحلية المطلة على البحر ، حيث يحوي العديد من وسائل الترفيه و الراحة مثل : الشاليهات ، وبرك السباحة ، و المرافق و الخدمات العامة بكافة أنواعها ، أماكن ممارسة الر...,منتجع شاطئ سيلين
4,و هو منتجع مطل على البحر ، و يحوي العديد من أماكن التنزه ، و يتميز بوفرة الشقق السكنية ، أخذ اسمه تبعا من المنطقة الموجود بها .,منتجع الغارية


In [11]:
arabic_data['articles'].iloc[3]

'يعتبر هذا المنتجع من أجمل الأماكن الساحلية المطلة على البحر ، حيث يحوي العديد من وسائل الترفيه و الراحة مثل : الشاليهات ، وبرك السباحة ، و المرافق و الخدمات العامة بكافة أنواعها ، أماكن ممارسة الرياضات البحرية و غيرها ، حيث يتميز موقع هذا المنتج بقربه من العاصمة ، حيث يبعد عنها مسافة ساعة بالسيارة . '

In [12]:
from keras import backend as K 
import gensim
from numpy import *
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
max_len_text=100
max_len_summary=10



In [13]:
from preprocessing import ArCleanText

In [14]:
ar_clean = ArCleanText()


 24%|██▍       | 57.5M/241M [00:06<00:21, 8.43MiB/s]

[2021-05-24 10:26:36,274 - farasapy_logger - ERROR]: an error occured
[2021-05-24 10:26:36,280 - farasapy_logger - ERROR]: File is not a zip file
[2021-05-24 10:26:36,284 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [15]:
cleaned_text = []
for t in arabic_data['articles']:
    cleaned_text.append(ar_clean.arabic_preprocessing(t))

In [16]:

#Call the above function
cleaned_summary = []
for t in arabic_data['summary']:
    cleaned_summary.append(ar_clean.arabic_preprocessing(t))

arabic_data['cleaned_text']=cleaned_text
arabic_data['cleaned_summary']=cleaned_summary
arabic_data['cleaned_summary'].replace('', np.nan, inplace=True)
arabic_data.dropna(axis=0,inplace=True)

In [17]:
arabic_data['cleaned_summary'] = arabic_data['cleaned_summary'].apply(lambda x : '_START_ '+ x + ' _END_')


In [18]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(arabic_data['cleaned_text'],arabic_data['cleaned_summary'],test_size=0.1,random_state=0,shuffle=True) 


In [19]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

thresh=4
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_len_text, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_len_text, padding='post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

print(x_voc)

% of rare words in vocabulary: 65.17279821627648
Total Coverage of rare words: 3.671436587435565
57795


In [20]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

thresh=6
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_len_summary, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_len_summary, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1

# y_tokenizer.word_counts['_START_'],len(y_tr)

% of rare words in vocabulary: 74.75019061760104
Total Coverage of rare words: 7.663174352154692


In [21]:
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)

ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

# **Next, let’s build the dictionary to convert the index to word for target and source vocabulary:**

In [22]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

# **We are defining a function below which is the implementation of the inference process**

In [23]:
from keras.models import  model_from_json
def load_model(model_filename, model_weights_filename):
    with open(model_filename, 'r', encoding='utf8') as f:
        model = model_from_json(f.read())
    model.load_weights(model_weights_filename)
    return model

encoder_testing = load_model('/content/drive/MyDrive/Freelancing/project 11/encoder_model.json', '/content/drive/MyDrive/Freelancing/project 11/encoder_model_weights.h5')
decoder_testing = load_model('/content/drive/MyDrive/Freelancing/project 11/decoder_model.json','/content/drive/MyDrive/Freelancing/project 11/decoder_model_weights.h5')

In [24]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_testing.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_testing.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='end'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'end'  or len(decoded_sentence.split()) >= (max_len_summary-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence
    
def seq2text(input_seq):
    newString=''
    for i in input_seq:
      if(i!=0):
        newString=newString+reverse_source_word_index[i]+' '
    
    return newString

In [25]:
def predict_summary(text):
  x_tokenizer = Tokenizer() 
  x_tokenizer.fit_on_texts(list(text))
  x_tr_seq    =   x_tokenizer.texts_to_sequences(text) 
  #padding zero upto maximum length
  x_test    =   pad_sequences(x_tr_seq,  maxlen=max_len_text, padding='post')
  for i in range(0,len(text)):
    print("Predicted summary:",decode_sequence(x_test[i].reshape(1,max_len_text)))



In [26]:
text = [' يحتوي الزنجبيل على مجموعة من المركبات الكيميائية التي تُساهم في عملية الهضم بشكلٍ عام، وتُساعد على تهدئة آلام المعدة']

In [27]:
predict_summary(text)

Predicted summary:  مدينه


In [28]:
text = ['احتوى عصير ليمون عديد فواءد بشر صفة معد خلال صفة مفيده يا ',
        'يعتبر هذا المنتجع من أجمل الأماكن الساحلية المطلة على البحر ، حيث يحوي العديد من وسائل الترفيه و الراحة مثل : الشاليهات ، وبرك السباحة ، و المرافق و الخدمات العامة بكافة أنواعها ، أماكن ممارسة الرياضات البحرية و غيرها ، حيث يتميز موقع هذا المنتج بقربه من العاصمة ، حيث يبعد عنها مسافة ساعة بالسيارة . ']

In [29]:
predict_summary(text)

Predicted summary:  فواءد التمر
Predicted summary:  مدينه
